# Introduction to the Reverse Osmosis Unit Model
This tutorial shows how to build, initialize, simulate, and optimize a reverse osmosis (RO) unit model using WaterTAP. In this example, we will minimize the specific energy consumption of the RO unit.

## Step 1: Import libraries from Pyomo, IDAES and WaterTAP.

In [ ]:
# Imports from Pyomo 
from pyomo.environ import (ConcreteModel, 
                           Objective, 
                           Expression, 
                           value) #including "value" for getting the value of Pyomo objects

# Imports from IDAES
from idaes.core import FlowsheetBlock     # Import flowsheet block from IDAES core
from watertap.core.solvers import get_solver    # Import function to get default solver
from idaes.core.util.model_statistics import degrees_of_freedom  # Import function to check degrees of freedom
from idaes.core.util.scaling import calculate_scaling_factors, set_scaling_factor  # Import utility function for calculating scaling factors


# Imports from WaterTAP
from watertap.property_models.NaCl_prop_pack import NaClParameterBlock   # Import NaCl property model
from watertap.unit_models.reverse_osmosis_0D import ReverseOsmosis0D     # Import RO model
from watertap.core import (ConcentrationPolarizationType, MassTransferCoefficient)


## Step 2: Start building the RO model. 
Create a Pyomo concrete model, flowsheet, and NaCl property parameter block. Add an RO unit to the flowsheet.



In [ ]:
m = ConcreteModel()
m.fs = FlowsheetBlock(dynamic=False)
m.fs.properties = NaClParameterBlock()

m.fs.unit = ReverseOsmosis0D(
    property_package=m.fs.properties,
    concentration_polarization_type=ConcentrationPolarizationType.none,
    mass_transfer_coefficient=MassTransferCoefficient.none,
    has_pressure_change=False,
    )

## Step 3: Specify values for system variables.

In [ ]:
m.fs.unit.inlet.flow_mass_phase_comp[0, 'Liq', 'NaCl'].fix(0.035)  # mass flow rate of NaCl (kg/s)
m.fs.unit.inlet.flow_mass_phase_comp[0, 'Liq', 'H2O'].fix(0.965)   # mass flow rate of water (kg/s)
m.fs.unit.inlet.pressure[0].fix(50e5)                              # feed pressure (Pa)
m.fs.unit.inlet.temperature[0].fix(298.15)                         # feed temperature (K)
m.fs.unit.area.fix(50)                                             # membrane area (m^2)
m.fs.unit.A_comp.fix(4.2e-12)                                      # membrane water permeability (m/Pa/s)
m.fs.unit.B_comp.fix(3.5e-8)                                       # membrane salt permeability (m/s)
m.fs.unit.permeate.pressure[0].fix(101325)                         # permeate pressure (Pa)

## Step 4: Scale all variables.
Set scaling factors for component mass flowrates, membrane area and all other variables.


In [ ]:
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1, index=('Liq', 'H2O'))
m.fs.properties.set_default_scaling('flow_mass_phase_comp', 1e2, index=('Liq', 'NaCl'))
set_scaling_factor(m.fs.unit.area, 1e-2)
calculate_scaling_factors(m)

## Step 5: Initialize the model.

In [ ]:
m.fs.unit.initialize()

## Step 6: Setup a solver and run a simulation.
Check that degrees of freedom = 0 before attempting simulation. This means that the performance of the flowsheet is completely determined by the system variables that were fixed above. Then run the simulation and report the results.

In [ ]:
assert degrees_of_freedom(m) == 0      # Check DOF
solver = get_solver()                  # Setup solver
simulation_results = solver.solve(m)   # Run simulation
m.fs.unit.report()                     # Display report, reports include a small subset of the most important variables

In [ ]:
# Display all results, this shows all variables and constraints
m.fs.unit.display()

## Step 7: Unfix variables (membrane area, feed pressure) and set variable bounds

In [ ]:
m.fs.unit.area.unfix()                  # unfix membrane area (m^2)
m.fs.unit.inlet.pressure[0].unfix()     # unfix feed pressure (Pa)

# Set lower and upper bounds for membrane area (m^2)
m.fs.unit.area.setlb(1)
m.fs.unit.area.setub(500)

# Set lower and upper bounds for feed pressure (Pa)
m.fs.unit.inlet.pressure[0].setlb(10e5)
m.fs.unit.inlet.pressure[0].setub(80e5)

## Step 8: Run optimization to minimize specific energy consumption

The optimization problem will find the membrane area and inlet pressure that achieve 50% recovery while minimizing specific energy consumption. 

In [ ]:
# Assume 100% efficiency of pumps and ERD and no pressure losses
#--> Pump power consumption ~ Qp*Pf/3.6e6
m.fs.specific_energy_consumption = Expression(
    expr=m.fs.unit.inlet.pressure[0]/(3.6e6))

# Define objective function to minimize the specific energy consumption.
m.fs.objective = Objective(expr=m.fs.specific_energy_consumption)

# Set the water recovery to 50%
m.fs.unit.recovery_vol_phase[0,'Liq'].fix(0.50)

# Check DOF 
print(degrees_of_freedom(m))

# Solve the model
optimization_results = solver.solve(m)
print(optimization_results)

# Display results of the optimized system
print(f"Optimized membrane area: {value(m.fs.unit.area)}")
print(f"Optimized membrane pressure: {value(m.fs.unit.inlet.pressure[0])}")
print(f"Specific energy consumption: {value(m.fs.specific_energy_consumption)}")
m.fs.unit.report()